# Vamos a leer el todoy generar una predicción basado en los trabajos de Diana

vamos por paso
Vamos a ver si por memoria tengo que partir

En este caso no voy a generar las covariables que generó Daiana

Voy a probar que corra, si corre luego le agrego las variables y veo los cambios en Kaggle




## Preparo todo

In [1]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


In [2]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend

import itertools

import gc
from datetime import datetime

#import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

In [3]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import matplotlib.pyplot as plt



### funciones Di

In [8]:
def percentage_safe(numerator: pd.Series, denominator: pd.Series, dtype='float32', fillna=None) -> pd.Series:
    """
    Calcula un porcentaje seguro como numerator / denominator.
    - Reemplaza divisiones por cero o NaN con NaN.
    - Opcionalmente convierte a float32.
    - Puede rellenar NaNs con `fillna`.
    """
    result = (numerator / denominator).mask((denominator == 0) | (denominator.isna()))
    if fillna is not None:
        result = result.fillna(fillna)
    return result.astype(dtype)

In [9]:
def reduce_mem_usage(df):
    """Itera por las columnas del DataFrame y modifica el tipo de datos para reducir uso de memoria."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Sólo convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(df[col]):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
    print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df

In [10]:
def clasificar_estado_producto(df: pd.DataFrame) -> None:
    """
    Clasifica el estado de los productos en un DataFrame basado en su ciclo de vida y ventas.

    La función evalúa una serie de condiciones y crea una nueva columna 'estado_producto'
    en el DataFrame proporcionado. La modificación se hace 'inplace'.

    Args:
        df (pd.DataFrame): El DataFrame que contiene los datos de los productos.
                           Debe incluir las columnas:
                           - 'meses_vida_producto'
                           - 'producto_crecimiento_ventas_suavizado'
                           - 'share_producto_en_categoria_suavizado'

    Returns:
        None: La función modifica el DataFrame directamente.
    """
    # Lista de condiciones en el orden de prioridad requerido
    condiciones = [
        (df['meses_vida_producto'] >= 0)& (df['meses_vida_producto'] <= 3),
        (df['producto_crecimiento_ventas_suavizado'] == 0),
        (df['producto_crecimiento_ventas_suavizado'] > 0.1) & (df['share_producto_en_categoria_suavizado'] > 0.1),
        (df['producto_crecimiento_ventas_suavizado'] < -0.1) & (df['share_producto_en_categoria_suavizado'] < -0.1)
    ]

    # Lista de resultados correspondientes a cada condición
    resultados = [
        'inicial',
        'sin ventas',
        'crecimiento',
        'contraccion'
    ]

    # El valor por defecto si ninguna condición se cumple es 'estable'
    df['estado_producto'] = np.select(condiciones, resultados, default='estable')

In [11]:
def calcular_ventanas_moviles(df, campo_base, prefijo_lag, tipo_agregacion, nombre_sufijo=None):
    """
    Calcula estadísticas móviles sobre ventanas específicas (3, 6 y 12 periodos).

    Parámetros:
    - df: DataFrame.
    - campo_base: nombre de la columna base (ej. "cat_total_tn").
    - prefijo_lag: parte de la cadena que compone la columna (ej. "_ma_").
    - tipo_agregacion: 'mean', 'std', 'min', 'max', 'median'.
    - nombre_sufijo: sufijo opcional para el nombre de la columna resultante.
    """
    if tipo_agregacion not in {"mean", "std", "min", "max", "median"}:
        raise ValueError("Agregación no soportada.")

    ventanas = [3, 6, 12]
    for ventana in ventanas:
        # Se genera el nombre completo de la columna que ya existe en el DataFrame.
        columna_ventana = f"{campo_base}{prefijo_lag}{ventana}"
        nombre_col = f"{campo_base}_{tipo_agregacion}_movil_{ventana}"
        if nombre_sufijo:
            nombre_col += f"_{nombre_sufijo}"
        # Aquí se aplica la agregación; en realidad, dado que se opera sobre una sola columna,
        # esto simplemente devolverá el mismo valor (o podría servir para dar formato).
        df[nombre_col] = getattr(df[[columna_ventana]], tipo_agregacion)(axis=1)

## Levanto el dataset
vamos a hacer el promedio de los últimos 12 meses por producto

In [18]:
# me traigo el archivo ya procesado con la duración completa
df_cluster_1 = pd.read_parquet(DATOS_DIR +'df_prod_base.parquet')
#df_cluster_3.head()
# productos a validar Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# me traigo el clasificador de clusters


In [19]:
# me quedo con las filas de df_cluster_3 cuyo product_id esté en productos_validar
###################################################
##### OJO, en realidad debo dejar todo
##### dejo las otras series que ayuden
##### saco de todo el conjunto lo que necesito al final
#############################
#df_cluster_X = df_cluster_3[df_cluster_3['product_id'].isin(productos_validar['product_id'])]
df_cluster_X = df_cluster_1.copy()
df_cluster_X.shape

(35888, 44)

In [20]:
df_cluster_X.shape

(35888, 44)

In [21]:
# tengo que pasar a timestamp todas las columnas con formato period[M] de df_cluster_X
# Identificamos automáticamente las columnas de tipo Period[M]
period_cols = df_cluster_X.select_dtypes(include=['period[M]']).columns

# Convertimos todas esas columnas a datetime usando .dt.to_timestamp()
df_cluster_X[period_cols] = df_cluster_X[period_cols].apply(lambda col: col.dt.to_timestamp())
df_cluster_X.dtypes

,0
product_id,int64
date_mes,datetime64[ns]
first_product_date,datetime64[ns]
last_product_date,datetime64[ns]
tn,float64
dif_tn,float64
dif_por,float64
vida_prod,int64
meses_on,int64
meses_off,int64


In [22]:
## A partir de df_cluster_X quiero generar un df_historia que tenga indice, primera columna produc_cust_id segunda columna el mínimo valor distinto de cero para dicho product_cust_id
# Filtra las filas donde tn es distinto de cero
df_filtrado = df_cluster_X[df_cluster_X['tn'] != 0]

# Agrupa por product_cust_id y calcula el mínimo de tn (distinto de cero)
df_historia = (
    df_filtrado
    .groupby('product_id', as_index=False)['tn']
    .min()
    .rename(columns={'tn': 'min'})
)

print(df_historia.head())

   product_id      min
0       20001  798.016
1       20002  505.886
2       20003  525.826
3       20004  415.525
4       20005  329.429


## agrando el df con lo realizado por Daiana / CHAT / di

In [23]:
df_cluster_X.columns

Index(['product_id', 'date_mes', 'first_product_date', 'last_product_date',
       'tn', 'dif_tn', 'dif_por', 'vida_prod', 'meses_on', 'meses_off',
       'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre', 'cat1',
       'cat2', 'cat3', 'brand', 'sku_size', 'cat1_cat2', 'cat1_cat2_cat3',
       'cat1_cat2_cat3_brand', 'cat1_cat2_cat3_brand_size', 'periodo',
       'stock_final', 'dias_laborables_arg', 'cisne_negro', 'VtaGral',
       'VtaGralCte', 'Bebidas', 'Almacén', 'Panadería', 'Lácteos', 'Carnes',
       'Verdulería y frutería', 'Artículos de limpieza y perfumería',
       'Indumentaria, calzado y textiles para el hogar',
       'Electrónicos y artículos para el hogar', 'Otros', 'USD_of', 'USD_blue',
       'Infl', 'Expect'],
      dtype='object')

#### le seguimos la corriente a Di, en algunos puntos

In [24]:
df_cluster_X.shape

(35888, 44)

In [ ]:
# Asumimos df_cluster_X con columnas: product_id, customer_id, date_mes (tipo datetime o Period), tn

# ---------- PASO 1: Agregar tn total por customer_id + date_mes
customer_monthly = (
    df_cluster_X.groupby(['customer_id', 'date_mes'], as_index=False)
    ['tn'].sum()
    .rename(columns={'tn': 'tn_sum_cus'})
)

# ---------- PASO 2: Agregar tn total por product_id + date_mes
product_monthly = (
    df_cluster_X.groupby(['product_id', 'date_mes'], as_index=False)
    ['tn'].sum()
    .rename(columns={'tn': 'tn_sum_prod'})
)

def calcular_estadisticas_adicionales(df, group_col, base_col):
    """
    En DataFrame con columnas (group_col, date_mes, base_col) calcula:
    - rolling mean (3,6,9,12)
    - lags (3,6,9,12)
    - deltas (con rolling y lag)
    - para cada métrica mensual base (tn_sum_*), rolling y lag adiciona min, max y std
    """
    ventana_list = [3,6,9,12]
    df = df.sort_values([group_col, 'date_mes']).copy()

    # Medias móviles rolling mean
    for ventana in ventana_list:
        col_rolling = f'{base_col}_rolling_{ventana}m'
        df[col_rolling] = (
            df.groupby(group_col)[base_col]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).mean())
        )

        # estadisticas rolling para las medias móviles
        df[f'{col_rolling}_min'] = (
            df.groupby(group_col)[col_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).min())
        )
        df[f'{col_rolling}_max'] = (
            df.groupby(group_col)[col_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).max())
        )
        df[f'{col_rolling}_std'] = (
            df.groupby(group_col)[col_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).std())
        )

    # Lags
    for lag in ventana_list:
        col_lag = f'{base_col}_lag_{lag}m'
        df[col_lag] = df.groupby(group_col)[base_col].shift(lag)

        # estadisticas para lags en ventana moviles
        df[f'{col_lag}_min'] = (
            df.groupby(group_col)[col_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).min())
        )
        df[f'{col_lag}_max'] = (
            df.groupby(group_col)[col_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).max())
        )
        df[f'{col_lag}_std'] = (
            df.groupby(group_col)[col_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).std())
        )

    # Deltas (con rolling y lag)
    for ventana in ventana_list:
        col_rolling = f'{base_col}_rolling_{ventana}m'
        col_delta_rolling = f'delta_rolling_{ventana}m'
        df[col_delta_rolling] = df[base_col] - df[col_rolling]

        # estadisticas para deltas rolling
        df[f'{col_delta_rolling}_min'] = (
            df.groupby(group_col)[col_delta_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).min())
        )
        df[f'{col_delta_rolling}_max'] = (
            df.groupby(group_col)[col_delta_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).max())
        )
        df[f'{col_delta_rolling}_std'] = (
            df.groupby(group_col)[col_delta_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).std())
        )

    for lag in ventana_list:
        col_lag = f'{base_col}_lag_{lag}m'
        col_delta_lag = f'delta_lag_{lag}m'
        df[col_delta_lag] = df[base_col] - df[col_lag]

        # estadisticas para deltas lag
        df[f'{col_delta_lag}_min'] = (
            df.groupby(group_col)[col_delta_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).min())
        )
        df[f'{col_delta_lag}_max'] = (
            df.groupby(group_col)[col_delta_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).max())
        )
        df[f'{col_delta_lag}_std'] = (
            df.groupby(group_col)[col_delta_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).std())
        )

    # También agregamos para base_col sin transformaciones estadísticas básicas por grupo y mes
    df[f'{base_col}_min'] = df.groupby(group_col)[base_col].transform('min')
    df[f'{base_col}_max'] = df.groupby(group_col)[base_col].transform('max')
    df[f'{base_col}_std'] = df.groupby(group_col)[base_col].transform('std')

    return df

# ---------- PASO 3: Calcular para customer_monthly
customer_monthly = calcular_estadisticas_adicionales(customer_monthly, 'customer_id', 'tn_sum_cus')

# ---------- PASO 4: Calcular para product_monthly
product_monthly = calcular_estadisticas_adicionales(product_monthly, 'product_id', 'tn_sum_prod')

# ---------- PASO 5: Unir métricas al DataFrame original

# Primero unimos tn_sum_cus + métricas customer
df_cluster_X = df_cluster_X.merge(
    customer_monthly,
    on=['customer_id', 'date_mes'],
    how='left',
    suffixes=('', '_cus')
)

# Luego unimos tn_sum_prod + métricas product
df_cluster_X = df_cluster_X.merge(
    product_monthly,
    on=['product_id', 'date_mes'],
    how='left',
    suffixes=('', '_prod')
)

# ---------- PASO 6: El DataFrame df_cluster_X tiene todas las métricas y estadísticas solicitadas
print(df_cluster_X.columns)

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por',
       ...
       'delta_lag_9m_min_prod', 'delta_lag_9m_max_prod',
       'delta_lag_9m_std_prod', 'delta_lag_12m_prod', 'delta_lag_12m_min_prod',
       'delta_lag_12m_max_prod', 'delta_lag_12m_std_prod', 'tn_sum_prod_min',
       'tn_sum_prod_max', 'tn_sum_prod_std'],
      dtype='object', length=181)


In [ ]:
df_cluster_X.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect,product_cust_id,tn_sum_cus,tn_sum_cus_rolling_3m,tn_sum_cus_rolling_3m_min,tn_sum_cus_rolling_3m_max,tn_sum_cus_rolling_3m_std,tn_sum_cus_rolling_6m,tn_sum_cus_rolling_6m_min,tn_sum_cus_rolling_6m_max,tn_sum_cus_rolling_6m_std,tn_sum_cus_rolling_9m,tn_sum_cus_rolling_9m_min,tn_sum_cus_rolling_9m_max,tn_sum_cus_rolling_9m_std,tn_sum_cus_rolling_12m,tn_sum_cus_rolling_12m_min,tn_sum_cus_rolling_12m_max,tn_sum_cus_rolling_12m_std,tn_sum_cus_lag_3m,tn_sum_cus_lag_3m_min,tn_sum_cus_lag_3m_max,tn_sum_cus_lag_3m_std,tn_sum_cus_lag_6m,tn_sum_cus_lag_6m_min,tn_sum_cus_lag_6m_max,tn_sum_cus_lag_6m_std,tn_sum_cus_lag_9m,tn_sum_cus_lag_9m_min,tn_sum_cus_lag_9m_max,tn_sum_cus_lag_9m_std,tn_sum_cus_lag_12m,tn_sum_cus_lag_12m_min,tn_sum_cus_lag_12m_max,tn_sum_cus_lag_12m_std,delta_rolling_3m,delta_rolling_3m_min,delta_rolling_3m_max,delta_rolling_3m_std,delta_rolling_6m,delta_rolling_6m_min,delta_rolling_6m_max,delta_rolling_6m_std,delta_rolling_9m,delta_rolling_9m_min,delta_rolling_9m_max,delta_rolling_9m_std,delta_rolling_12m,delta_rolling_12m_min,delta_rolling_12m_max,delta_rolling_12m_std,delta_lag_3m,delta_lag_3m_min,delta_lag_3m_max,delta_lag_3m_std,delta_lag_6m,delta_lag_6m_min,delta_lag_6m_max,delta_lag_6m_std,delta_lag_9m,delta_lag_9m_min,delta_lag_9m_max,delta_lag_9m_std,delta_lag_12m,delta_lag_12m_min,delta_lag_12m_max,delta_lag_12m_std,tn_sum_cus_min,tn_sum_cus_max,tn_sum_cus_std,tn_sum_prod,tn_sum_prod_rolling_3m,tn_sum_prod_rolling_3m_min,tn_sum_prod_rolling_3m_max,tn_sum_prod_rolling_3m_std,tn_sum_prod_rolling_6m,tn_sum_prod_rolling_6m_min,tn_sum_prod_rolling_6m_max,tn_sum_prod_rolling_6m_std,tn_sum_prod_rolling_9m,tn_sum_prod_rolling_9m_min,tn_sum_prod_rolling_9m_max,tn_sum_prod_rolling_9m_std,tn_sum_prod_rolling_12m,tn_sum_prod_rolling_12m_min,tn_sum_prod_rolling_12m_max,tn_sum_prod_rolling_12m_std,tn_sum_prod_lag_3m,tn_sum_prod_lag_3m_min,tn_sum_prod_lag_3m_max,tn_sum_prod_lag_3m_std,tn_sum_prod_lag_6m,tn_sum_prod_lag_6m_min,tn_sum_prod_lag_6m_max,tn_sum_prod_lag_6m_std,tn_sum_prod_lag_9m,tn_sum_prod_lag_9m_min,tn_sum_prod_lag_9m_max,tn_sum_prod_lag_9m_std,tn_sum_prod_lag_12m,tn_sum_prod_lag_12m_min,tn_sum_prod_lag_12m_max,tn_sum_prod_lag_12m_std,delta_rolling_3m_prod,delta_rolling_3m_min_prod,delta_rolling_3m_max_prod,delta_rolling_3m_std_prod,delta_rolling_6m_prod,delta_rolling_6m_min_prod,delta_rolling_6m_max_prod,delta_rolling_6m_std_prod,delta_rolling_9m_prod,delta_rolling_9m_min_prod,delta_rolling_9m_max_prod,delta_rolling_9m_std_prod,delta_rolling_12m_prod,delta_rolling_12m_min_prod,delta_rolling_12m_max_prod,delta_rolling_12m_std_prod,delta_lag_3m_prod,delta_lag_3m_min_prod,delta_lag_3m_max_prod,delta_lag_3m_std_prod,delta_lag_6m_prod,delta_lag_6m_min_prod,delta_lag_6m_max_prod,delta_lag_6m_std_prod,delta_lag_9m_prod,delta_lag_9m_min_prod,delta_lag_9m_max_prod,delta_lag_9m_std_prod,delta_lag_12m_prod,delta_lag_12m_min_prod,delta_lag_12m_max_prod,delta_lag_12m_std_prod,tn_sum_prod_min,tn_sum_prod_max,tn_sum_prod_std
0,10001,20002,2017-01-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,87.649,-2.486,-0.028,4,4,4,0,60,2017,1,1,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_LIMPIEX,HC_ROPA LAVADO_Liquido_LIMPIEX_3000.0,201701,NaN,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47,20002_10001,123.853,123.853,123.853,123.853,NaN,123.853,123.853,123.

In [ ]:
df_cluster_X.shape

(236511, 181)

In [25]:
# lgbm no banca tildes ni espacios en los nombres de columna
# Diccionario de reemplazos para vocales con tilde
replacements = {
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'Á': 'A',
    'É': 'E',
    'Í': 'I',
    'Ó': 'O',
    'Ú': 'U',
}

def clean_column_names(col_names):
    cleaned_cols = []
    for col in col_names:
        # Reemplazar espacios por guion bajo
        col = col.replace(' ', '_')
        # Eliminar comas
        col = col.replace(',', '')
        # Reemplazar vocales con tilde
        for orig, repl in replacements.items():
            col = col.replace(orig, repl)
        cleaned_cols.append(col)
    return cleaned_cols

# Aplica al DataFrame df_cluster_X
df_cluster_X.columns = clean_column_names(df_cluster_X.columns)

# Comprobar resultado
print(df_cluster_X.columns)

Index(['product_id', 'date_mes', 'first_product_date', 'last_product_date',
       'tn', 'dif_tn', 'dif_por', 'vida_prod', 'meses_on', 'meses_off',
       'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre', 'cat1',
       'cat2', 'cat3', 'brand', 'sku_size', 'cat1_cat2', 'cat1_cat2_cat3',
       'cat1_cat2_cat3_brand', 'cat1_cat2_cat3_brand_size', 'periodo',
       'stock_final', 'dias_laborables_arg', 'cisne_negro', 'VtaGral',
       'VtaGralCte', 'Bebidas', 'Almacen', 'Panaderia', 'Lacteos', 'Carnes',
       'Verduleria_y_fruteria', 'Articulos_de_limpieza_y_perfumeria',
       'Indumentaria_calzado_y_textiles_para_el_hogar',
       'Electronicos_y_articulos_para_el_hogar', 'Otros', 'USD_of', 'USD_blue',
       'Infl', 'Expect'],
      dtype='object')


In [27]:
# vamos a dejar un product_id_str
df_cluster_X['product_id_str'] = df_cluster_X['product_id'].astype(str)

In [28]:
df_cluster_X.dtypes

,0
product_id,int64
date_mes,datetime64[ns]
first_product_date,datetime64[ns]
last_product_date,datetime64[ns]
tn,float64
dif_tn,float64
dif_por,float64
vida_prod,int64
meses_on,int64
meses_off,int64


In [33]:
# vamos a hacer featur con ideas Di plus Chat
## esto hay que optimizarlo. Lo corté antes por tiempo
df = df_cluster_X.copy()
#df['date_mes'] = pd.to_datetime(df['date_mes'])
df = df.sort_values(['product_id', 'date_mes']).reset_index(drop=True)

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'product_id' in num_cols:
    num_cols.remove('product_id')

rolling_windows = [2, 3, 6, 9, 12]
max_lag = 12

for col in num_cols:
    # Lags
    for lag in range(1, max_lag + 1):
        df[f'{col}_lag{lag}'] = df.groupby('product_id')[col].shift(lag)
    # Rolling aggregations
    for win in rolling_windows:
        roll_grp = df.groupby('product_id')[col].rolling(win, min_periods=1)
        df[f'{col}_roll{win}_mean'] = roll_grp.mean().reset_index(level=0, drop=True)
        df[f'{col}_roll{win}_min'] = roll_grp.min().reset_index(level=0, drop=True)
        df[f'{col}_roll{win}_max'] = roll_grp.max().reset_index(level=0, drop=True)
        df[f'{col}_roll{win}_std'] = roll_grp.std().reset_index(level=0, drop=True)

        # Meses desde el máximo/mínimo de la ventana rolling
        def months_since_max(x):
            if x.isnull().all():
                return np.nan
            idx = np.argmax(x[::-1])  # posición desde el final de la ventana
            return idx

        def months_since_min(x):
            if x.isnull().all():
                return np.nan
            idx = np.argmin(x[::-1])
            return idx

        df[f'{col}_roll{win}_months_since_max'] = roll_grp.apply(months_since_max, raw=False).reset_index(level=0, drop=True)
        df[f'{col}_roll{win}_months_since_min'] = roll_grp.apply(months_since_min, raw=False).reset_index(level=0, drop=True)

        # Tendencia (pendiente, robusta a NaNs)
        def rolling_slope_clean(x):
            # Devuelve NaN si hay NaN en la ventana o menos de 2 datos
            if x.isnull().any() or len(x.dropna()) < 2:
                return np.nan
            y = x.values
            X = np.arange(len(y)).reshape(-1, 1)
            model = LinearRegression().fit(X, y)
            return model.coef_[0]

        df[f'{col}_roll{win}_trend'] = (
            df.groupby('product_id')[col]
            .rolling(win, min_periods=2)
            .apply(rolling_slope_clean, raw=False)
            .reset_index(level=0, drop=True)
        )

# Defragmentar el dataframe (opcional pero recomendado después de muchos agregados)
df = df.copy()
df.shape

/tmp/ipython-input-33-2110557375.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df.groupby('product_id')[col].shift(lag)
/tmp/ipython-input-33-2110557375.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df.groupby('product_id')[col].shift(lag)
/tmp/ipython-input-33-2110557375.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

KeyboardInterrupt: 

In [34]:
df = df.copy()
df.shape

(35888, 1243)

In [35]:
# Lista de columnas de categoría según lo indicado
cat_cols = [
    'cat1', 'cat2', 'cat3', 'brand',
    'cat1_cat2', 'cat1_cat2_cat3',
    'cat1_cat2_cat3_brand',
    'cat1_cat2_cat3_brand_size'
]

# Asegura fechas en formato datetime y orden
#df['date_mes'] = pd.to_datetime(df['date_mes'])
df = df.sort_values(['date_mes', 'product_id']).reset_index(drop=True)

# Total de ventas por mes (en toda la tabla)
df['ventas_total_mes'] = df.groupby('date_mes')['tn'].transform('sum')

# Para cada categórica: features por mes
for cat in cat_cols:
    # Ventas de la categoría en ese mes
    print(cat)
    col_vtas_cat_mes = f'{cat}_ventas_mes'
    df[col_vtas_cat_mes] = df.groupby(['date_mes', cat])['tn'].transform('sum')

    # % de ventas categoría / ventas totales del mes
    col_pct_cat_mes = f'{cat}_pct_mes'
    df[col_pct_cat_mes] = df[col_vtas_cat_mes] / df['ventas_total_mes']

    # Para cada producto asociado a la categoría: % producto/total de categoría en el mes
    col_cat_prod_vtas = f'{cat}_prod_ventas_mes'
    col_pct_prod_cat_mes = f'{cat}_prod_pct_cat_mes'
    df[col_cat_prod_vtas] = df.groupby(['date_mes', cat, 'product_id'])['tn'].transform('sum')
    # % ventas producto en la cat/ventas totales cat ese mes
    df[col_pct_prod_cat_mes] = df[col_cat_prod_vtas] / df[col_vtas_cat_mes]

    # Para cada producto: % producto/total ventas en el mes
    col_pct_prod_mes = f'{cat}_prod_pct_mes'
    df[col_pct_prod_mes] = df[col_cat_prod_vtas] / df['ventas_total_mes']

    # Opcional: elimina columna auxiliar temporal
    # df.drop(col_cat_prod_vtas, axis=1, inplace=True)

    # Lags y Rolling: para cada métrica de la categórica
    for ftr in [col_vtas_cat_mes, col_pct_cat_mes, col_pct_prod_cat_mes, col_pct_prod_mes]:
        # Lags de 1-12 meses por categoría/por producto
        for lag in range(1, 13):
            df[f'{ftr}_lag{lag}'] = (
                df.groupby(cat)[ftr].shift(lag)
                if 'prod' not in ftr
                else df.groupby([cat, 'product_id'])[ftr].shift(lag)
            )
        # Rolling por categoría/por producto
        for win in [3, 6, 9, 12]:
            # Rolling por categoría:
            if 'prod' not in ftr:
                gb = df.groupby(cat)[ftr]
                df[f'{ftr}_roll{win}_mean'] = gb.rolling(win, min_periods=1).mean().reset_index(level=0, drop=True)
                df[f'{ftr}_roll{win}_max'] = gb.rolling(win, min_periods=1).max().reset_index(level=0, drop=True)
                df[f'{ftr}_roll{win}_std'] = gb.rolling(win, min_periods=1).std().reset_index(level=0, drop=True)
            else:
                gb = df.groupby([cat, 'product_id'])[ftr]
                df[f'{ftr}_roll{win}_mean'] = gb.rolling(win, min_periods=1).mean().reset_index(level=[0,1], drop=True)
                df[f'{ftr}_roll{win}_max'] = gb.rolling(win, min_periods=1).max().reset_index(level=[0,1], drop=True)
                df[f'{ftr}_roll{win}_std'] = gb.rolling(win, min_periods=1).std().reset_index(level=[0,1], drop=True)

# Limpia bloques de memoria fragmentados
df = df.copy()
df.shape

cat1


/tmp/ipython-input-35-1780402858.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{ftr}_roll{win}_max'] = gb.rolling(win, min_periods=1).max().reset_index(level=[0,1], drop=True)
/tmp/ipython-input-35-1780402858.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{ftr}_roll{win}_std'] = gb.rolling(win, min_periods=1).std().reset_index(level=[0,1], drop=True)
/tmp/ipython-input-35-1780402858.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

cat2


/tmp/ipython-input-35-1780402858.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{ftr}_roll{win}_std'] = gb.rolling(win, min_periods=1).std().reset_index(level=0, drop=True)
/tmp/ipython-input-35-1780402858.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{ftr}_lag{lag}'] = (
/tmp/ipython-input-35-1780402858.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

cat3


...

cat1_cat2_cat3_brand


/tmp/ipython-input-35-1780402858.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{ftr}_roll{win}_max'] = gb.rolling(win, min_periods=1).max().reset_index(level=0, drop=True)
/tmp/ipython-input-35-1780402858.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{ftr}_roll{win}_std'] = gb.rolling(win, min_periods=1).std().reset_index(level=0, drop=True)
/tmp/ipython-input-35-1780402858.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

cat1_cat2_cat3_brand_size


/tmp/ipython-input-35-1780402858.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{ftr}_roll{win}_mean'] = gb.rolling(win, min_periods=1).mean().reset_index(level=0, drop=True)
/tmp/ipython-input-35-1780402858.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{ftr}_roll{win}_max'] = gb.rolling(win, min_periods=1).max().reset_index(level=0, drop=True)
/tmp/ipython-input-35-1780402858.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which 

(35888, 2052)

In [37]:
df.shape

(35888, 2052)

In [38]:
# como el trabajo ha sido interesante guardo el dataset luego tendré solo normalizar y calcular.. solo
df.to_parquet(DATOS_DIR +'df_prod_completo_full.parquet', index=False)